### Подключение к базе и основные таблицы

In [1]:
import pandas as pd

### Данные по пользователям

user_info = pd.read_sql(
    """SELECT * FROM public.user_data""",
    
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

user_info.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [2]:
### Посты и топики

posts_info = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

posts_info.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [3]:
### Почти 77 миллионов данных, достаточно тяжело
### Попробуем забрать, скажем, 10 миллионов

feed_data = pd.read_sql(
    """SELECT * FROM public.feed_data LIMIT 10000000""",
    
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-11-05 19:07:31,80072,1559,like,0
1,2021-11-05 19:07:33,80072,5040,view,0
2,2021-11-05 19:08:59,80072,1566,view,1
3,2021-11-05 19:09:23,80072,1566,like,0
4,2021-11-05 19:09:25,80072,3017,view,0


### Работа с данными и фичи для контентной модели

In [4]:
### Как устроена колонка target?

feed_data[feed_data.action!='view']

,timestamp,user_id,post_id,action,target
0,2021-11-05 19:07:31,80072,1559,like,0
3,2021-11-05 19:09:23,80072,1566,like,0
11,2021-11-05 19:23:29,80072,3569,like,0
13,2021-11-05 19:26:24,80072,6378,like,0
25,2021-11-05 19:44:06,80072,5736,like,0
...,...,...,...,...,...
9999896,2021-12-15 14:11:31,109973,1117,like,0
9999903,2021-12-15 14:22:54,109973,436,like,0
9999932,2021-12-16 15:58:00,109973,15,like,0
9999955,2021-12-18 08:00:09,109973,1610,like,0


In [5]:
### Почистим данные от action != view

feed_data = feed_data[feed_data.action == 'view']

feed_data.head()

,timestamp,user_id,post_id,action,target
1,2021-11-05 19:07:33,80072,5040,view,0
2,2021-11-05 19:08:59,80072,1566,view,1
4,2021-11-05 19:09:25,80072,3017,view,0
5,2021-11-05 19:11:15,80072,1365,view,0
6,2021-11-05 19:14:10,80072,2316,view,0


Напомним, как устроен контентный подход! 

Научимся по выбранной дате timestamp 

И для любой пары (user_id, post_id)

Предсказывать, случится лайк или нет

Хорошо бы иметь модель, которая умеет предсказывать вероятности

In [11]:
### Сделаем эмбеддинги постов
from transformers import AutoTokenizer
from transformers import BertModel  
from transformers import RobertaModel  
from transformers import DistilBertModel  


def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',  
        'roberta': 'roberta-base',  
        'distilbert': 'distilbert-base-cased'  
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)


In [12]:
tokenizer, model = get_model('distilbert')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

C:\Users\OMEN\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\OMEN\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
### Сделаем датасет для постов

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding


class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        super().__init__()

        self.texts = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {'input_ids': self.texts['input_ids'][idx], 'attention_mask': self.texts['attention_mask'][idx]}

    def __len__(self):
        return len(self.texts['input_ids'])
    
    
dataset = PostDataset(posts_info['text'].values.tolist(), tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

b = next(iter(loader))

b

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  101,  1993,  4190,  ...,     0,     0,     0],
        [  101, 15386,  1116,  ...,  1300,  1107,   102],
        [  101,  3141,   186,  ..., 14099,  8478,   102],
        ...,
        [  101, 16972, 20647,  ...,     0,     0,     0],
        [  101,   137,   188,  ...,     0,     0,     0],
        [  101,   144, 22731,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [14]:
import torch
from tqdm import tqdm


@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [18]:
embeddings = get_embeddings_labels(model, loader).numpy()

embeddings

array([[ 3.63150924e-01,  4.89375107e-02, -2.64081419e-01, ...,
        -1.41593292e-01,  1.59181058e-02,  9.18474980e-05],
       [ 2.36416146e-01, -1.59501195e-01, -3.27798426e-01, ...,
        -2.89936155e-01,  1.19365156e-01, -1.62365846e-03],
       [ 3.75191331e-01, -1.13943920e-01, -2.40547195e-01, ...,
        -3.38919163e-01,  5.86937591e-02, -2.12656315e-02],
       ...,
       [ 3.40382338e-01,  6.64923489e-02, -1.63184568e-01, ...,
        -8.65626335e-02,  2.03403682e-01,  3.20905708e-02],
       [ 4.32092071e-01,  1.10913180e-02, -1.17306173e-01, ...,
         7.54014030e-02,  1.02739736e-01,  1.52741875e-02],
       [ 3.04277599e-01, -7.62156248e-02, -6.77586049e-02, ...,
        -5.43486886e-02,  2.44383708e-01, -1.41484234e-02]], dtype=float32)

In [17]:
### Пытаемся кластеризовать тексты

from sklearn.decomposition import PCA

centered = embeddings - embeddings.mean()

pca = PCA(n_components=20)
pca_decomp = pca.fit_transform(centered)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=15, random_state=0).fit(pca_decomp)

posts_info['TextCluster'] = kmeans.labels_

dists_columns = ['DistanceTo1thCluster',
                 'DistanceTo2thCluster',
                 'DistanceTo3thCluster',
                 'DistanceTo4thCluster',
                 'DistanceTo5thCluster',
                 'DistanceTo6thCluster',
                 'DistanceTo7thCluster',
                 'DistanceTo8thCluster',
                 'DistanceTo9thCluster',
                 'DistanceTo10thCluster',
                 'DistanceTo11thCluster',
                 'DistanceTo12thCluster',
                 'DistanceTo13thCluster',
                 'DistanceTo14thCluster',
                 'DistanceTo15thCluster']

dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,3.304246,3.620401,2.859321,3.298448,2.694607,3.270909,2.033472,1.734975,3.228055,3.366776,2.164244,3.342857,1.492624,3.497085,3.299263
1,3.184774,3.331278,2.657021,3.040497,2.359118,3.238412,1.977695,1.167934,3.178972,3.100411,2.096817,2.984859,1.903261,3.193114,2.773982
2,3.233075,3.411761,2.934810,3.130858,2.750259,3.396074,2.893854,1.425920,3.124659,3.291293,2.204940,3.017749,1.632557,3.213984,2.776564
3,3.605784,2.886860,3.128229,3.553193,3.246159,3.609459,3.255307,2.145011,3.372815,3.938695,2.617186,3.666022,2.301664,3.649338,3.561258
4,2.591213,3.073209,2.429409,2.645641,1.824097,2.575958,2.734103,1.701688,2.826227,3.017270,1.572849,2.539641,1.163709,2.811826,2.419337


In [19]:
posts_info = pd.concat((posts_info, dists_df), axis=1)

posts_info.head()

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,12,3.304246,3.620401,2.859321,3.298448,2.694607,3.270909,2.033472,1.734975,3.228055,3.366776,2.164244,3.342857,1.492624,3.497085,3.299263
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,7,3.184774,3.331278,2.657021,3.040497,2.359118,3.238412,1.977695,1.167934,3.178972,3.100411,2.096817,2.984859,1.903261,3.193114,2.773982
2,3,Asian quake hits European shares\n\nShares in ...,business,7,3.233075,3.411761,2.934810,3.130858,2.750259,3.396074,2.893854,1.425920,3.124659,3.291293,2.204940,3.017749,1.632557,3.213984,2.776564
3,4,India power shares jump on debut\n\nShares in ...,business,7,3.605784,2.886860,3.128229,3.553193,3.246159,3.609459,3.255307,2.145011,3.372815,3.938695,2.617186,3.666022,2.301664,3.649338,3.561258
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,12,2.591213,3.073209,2.429409,2.645641,1.824097,2.575958,2.734103,1.701688,2.826227,3.017270,1.572849,2.539641,1.163709,2.811826,2.419337


In [22]:
posts_info['topic'] = posts_info['topic'].map(pd.merge(
            feed_data,
            posts_info.drop('text', axis=1),
            on='post_id',
            how='left').groupby('topic')['target'].mean())

In [23]:
posts_info.to_sql(    
   "posts_info_features",                    
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml",                      
    schema="public",                   
    if_exists='replace'            
   ) 

23

In [24]:
#поработаем с фичами юзеров
test_ = pd.read_sql(
    """SELECT * FROM public.user_info_features""",
    
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)
test_ = test_.drop(['index', 'countoflike'])

,index,user_id,gender,srmap,countoflike,like_0,like_1,like_2,age,city,exp_group
0,0,200,1,0.148193,358,6,1,3,34,Degtyarsk,3
1,1,201,0,0.069763,690,6,1,5,37,Abakan,0
2,2,202,1,0.111084,637,6,1,4,17,Smolensk,4
3,3,203,0,0.186401,321,6,1,3,18,Moscow,1
4,4,204,0,0.000000,138,6,1,3,36,Anzhero-Sudzhensk,3
...,...,...,...,...,...,...,...,...,...,...,...
163200,163200,168548,0,0.000000,362,6,1,5,36,Kaliningrad,4
163201,163201,168549,0,0.000000,252,4,6,1,18,Tula,2
163202,163202,168550,1,0.071411,369,6,1,5,41,Yekaterinburg,4
163203,163203,168551,0,0.152954,477,6,1,3,38,Moscow,3


In [48]:
user_info = test_.drop(['index', 'countoflike'], axis=1)

In [50]:
user_info['city'] = user_info['city'].map(pd.merge(
            feed_data,
            user_info[['user_id', 'city']],
            on='user_id',
            how='left').groupby('city')['target'].mean())

In [52]:
user_info['exp_group'] = user_info['exp_group'].map(pd.merge(
            feed_data,
            user_info[['user_id', 'exp_group']],
            on='user_id',
            how='left').groupby('exp_group')['target'].mean())

In [53]:
user_info

,user_id,gender,srmap,like_0,like_1,like_2,age,city,exp_group
0,200,1,0.148193,6,1,3,34,0.157534,0.093361
1,201,0,0.069763,6,1,5,37,0.129490,0.094053
2,202,1,0.111084,6,1,4,17,0.149247,0.112245
3,203,0,0.186401,6,1,3,18,0.155936,0.148811
4,204,0,0.000000,6,1,3,36,0.146527,0.093361
...,...,...,...,...,...,...,...,...,...
163200,168548,0,0.000000,6,1,5,36,0.067210,0.112245
163201,168549,0,0.000000,4,6,1,18,0.087285,0.148441
163202,168550,1,0.071411,6,1,5,41,0.079892,0.112245
163203,168551,0,0.152954,6,1,3,38,0.155936,0.093361


In [54]:
user_info.to_sql(    
   "user_info_features",                    
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml",                      
    schema="public",                   
    if_exists='replace'            
   ) 

205

In [55]:
### Воспроизведем датафрейм со всеми новыми фичами 

df = pd.merge(feed_data,
              posts_info,
              on='post_id',
              how='left')

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,...,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,2021-11-05 19:07:33,80072,5040,view,0,"Not only was this film exhilarating, but it to...",0.121222,0,1.312348,2.502413,...,1.635818,3.253526,2.887747,2.684970,3.145558,2.531785,1.735963,2.833629,2.200534,2.695899
1,2021-11-05 19:08:59,80072,1566,view,1,Kerr frustrated at victory margin\n\nRepublic ...,0.123896,9,3.455095,3.807062,...,3.527050,3.078518,3.632820,3.974306,0.876688,3.486864,3.251134,3.379247,3.712589,2.675395
2,2021-11-05 19:09:25,80072,3017,view,0,Gut-wrenching &amp; heart broken. Lost three m...,0.119944,13,2.259625,2.787625,...,2.593166,3.348488,3.144202,2.142853,3.397564,2.829831,2.501806,3.169005,1.247586,2.948416
3,2021-11-05 19:11:15,80072,1365,view,0,Milburn defends poster campaign\n\nLabours ele...,0.113412,6,3.534899,3.897914,...,3.617672,1.056810,2.897252,3.896637,3.149810,2.918094,3.475394,2.878770,3.760947,3.478173
4,2021-11-05 19:14:10,80072,2316,view,0,Loyalty cards idea for TV addicts\n\nViewers c...,0.098921,10,2.439560,2.724213,...,2.627032,1.806946,2.207119,3.098247,2.830954,1.281974,2.455299,1.720792,2.905696,2.895061


In [56]:
df = pd.merge(df,
              user_info,
              on='user_id',
              how='left')

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,...,DistanceTo14thCluster,DistanceTo15thCluster,gender,srmap,like_0,like_1,like_2,age,city,exp_group
0,2021-11-05 19:07:33,80072,5040,view,0,"Not only was this film exhilarating, but it to...",0.121222,0,1.312348,2.502413,...,2.200534,2.695899,1,0.231689,6,1,3,27,0.154716,0.093361
1,2021-11-05 19:08:59,80072,1566,view,1,Kerr frustrated at victory margin\n\nRepublic ...,0.123896,9,3.455095,3.807062,...,3.712589,2.675395,1,0.231689,6,1,3,27,0.154716,0.093361
2,2021-11-05 19:09:25,80072,3017,view,0,Gut-wrenching &amp; heart broken. Lost three m...,0.119944,13,2.259625,2.787625,...,1.247586,2.948416,1,0.231689,6,1,3,27,0.154716,0.093361
3,2021-11-05 19:11:15,80072,1365,view,0,Milburn defends poster campaign\n\nLabours ele...,0.113412,6,3.534899,3.897914,...,3.760947,3.478173,1,0.231689,6,1,3,27,0.154716,0.093361
4,2021-11-05 19:14:10,80072,2316,view,0,Loyalty cards idea for TV addicts\n\nViewers c...,0.098921,10,2.439560,2.724213,...,2.905696,2.895061,1,0.231689,6,1,3,27,0.154716,0.093361


In [57]:
### А еще научимся выделять признаки
### Из timestamp!
### от времени просмотра может зависеть
### Склонность пользователей лайкать или игнорировать посты

df['hour'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.hour)
df['month'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.month)

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,...,gender,srmap,like_0,like_1,like_2,age,city,exp_group,hour,month
0,2021-11-05 19:07:33,80072,5040,view,0,"Not only was this film exhilarating, but it to...",0.121222,0,1.312348,2.502413,...,1,0.231689,6,1,3,27,0.154716,0.093361,19,11
1,2021-11-05 19:08:59,80072,1566,view,1,Kerr frustrated at victory margin\n\nRepublic ...,0.123896,9,3.455095,3.807062,...,1,0.231689,6,1,3,27,0.154716,0.093361,19,11
2,2021-11-05 19:09:25,80072,3017,view,0,Gut-wrenching &amp; heart broken. Lost three m...,0.119944,13,2.259625,2.787625,...,1,0.231689,6,1,3,27,0.154716,0.093361,19,11
3,2021-11-05 19:11:15,80072,1365,view,0,Milburn defends poster campaign\n\nLabours ele...,0.113412,6,3.534899,3.897914,...,1,0.231689,6,1,3,27,0.154716,0.093361,19,11
4,2021-11-05 19:14:10,80072,2316,view,0,Loyalty cards idea for TV addicts\n\nViewers c...,0.098921,10,2.439560,2.724213,...,1,0.231689,6,1,3,27,0.154716,0.093361,19,11


In [58]:
### Уберем все ненужные колонки

df = df.drop([
    'action',
    'text',
],
    axis=1)

df = df.set_index(['user_id', 'post_id'])

df.head(50)

timestamp  target     topic  TextCluster  \
user_id post_id                                                      
80072   5040    2021-11-05 19:07:33       0  0.121222            0   
        1566    2021-11-05 19:08:59       1  0.123896            9   
        3017    2021-11-05 19:09:25       0  0.119944           13   
        1365    2021-11-05 19:11:15       0  0.113412            6   
        2316    2021-11-05 19:14:10       0  0.098921           10   
        6650    2021-11-05 19:15:56       0  0.121222            0   
        3499    2021-11-05 19:17:52       0  0.119944           13   
        4979    2021-11-05 19:18:46       0  0.121222            5   
        3569    2021-11-05 19:20:46       1  0.119944           13   
        6378    2021-11-05 19:23:31       1  0.121222            0   
        2798    2021-11-05 19:26:26       0  0.119944            2   
        3654    2021-11-05 19:29:19       0  0.119944           13   
        2128    2021-11-05 19:30:20       0  0.098921           10   
        6196    2021-11-05 19:30:53       0  0.121222            0   
        6279    2021-11-05 19:32:24       0  0.121222            5   
        3314    2021-11-05 19:35:12       0  0.119944            8   
        2642    2021-11-05 19:36:27       0  0.119944            2   
        1736    2021-11-05 19:37:51       0  0.123896            9   
        4047    2021-11-05 19:39:04       0  0.119944           13   
        6638    2021-11-05 19:41:49       0  0.121222            5   
        5736    2021-11-05 19:42:12       1  0.121222            0   
        6478    2021-11-05 19:44:08       0  0.121222            1   
        1526    2021-11-05 19:46:00       0  0.123896            9   
        1581    2021-11-05 19:47:40       0  0.123896            9   
        4897    2021-11-05 19:48:25       0  0.121222            5   
        4189    2021-11-06 07:14:12       0  0.119944            2   
        1851    2021-11-06 07:15:41       0  0.123896            9   
        1531    2021-11-06 07:18:39       0  0.123896            9   
        1378    2021-11-06 07:21:28       0  0.113412            6   
        855     2021-11-06 07:23:49       0  0.123080            4   
        1158    2021-11-06 07:26:16       0  0.113412            6   
        1838    2021-11-06 07:27:35       0  0.123896            9   
        6282    2021-11-06 07:28:27       1  0.121222           11   
        5990    2021-11-06 07:28:40       0  0.121222            5   
        2588    2021-11-06 07:31:22       1  0.119944            8   
        414     2021-11-06 07:33:52       0  0.119098            7   
        5996    2021-11-06 07:35:41       0  0.121222            0   
        3066    2021-11-06 07:38:30       0  0.119944            2   
        115     2021-11-06 07:40:50       0  0.119098           10   
        656     2021-11-06 07:43:37       0  0.123080            4   
        1776    2021-11-06 07:44:54       0  0.123896           14   
        1648    2021-11-06 07:47:13       0  0.123896            9   
        1743    2021-11-06 07:47:39       1  0.123896            9   
        790     2021-11-06 07:49:19       0  0.123080            4   
        2234    2021-11-06 07:51:36       0  0.098921           10   
        4527    2021-11-06 07:52:45       0  0.121222            3   
        6577    2021-11-06 07:55:40       0  0.121222            5   
        6377    2021-11-06 07:56:03       0  0.121222            5   
        5609    2021-11-06 07:57:55       0  0.121222            0   
        4957    2021-11-06 08:00:23       0  0.121222            0   

                 DistanceTo1thCluster  DistanceTo2thCluster  \
user_id post_id                                               
80072   5040                 1.312348              2.502413   
        1566                 3.455095              3.807062   
        3017                 2.259625              2.787625   
        1365                 3.534899              3.897914   
       

### Пора обучать модели! 

In [59]:
X = df.drop(['target', 'timestamp'], axis=1)
Y = df['target']

In [60]:
### Обучим катбуст!

from catboost import CatBoostClassifier


catboost = CatBoostClassifier(
    iterations=200,
    learning_rate=1,
    depth=2,
    random_seed=12345
)

object_cols = [
    'TextCluster', 'gender', 'age', 'hour', 'month',
]

catboost.fit(X, Y, object_cols)

0:	learn: 0.3577820	total: 4.59s	remaining: 15m 13s
1:	learn: 0.3556815	total: 10.1s	remaining: 16m 42s
2:	learn: 0.3544167	total: 14.3s	remaining: 15m 36s
3:	learn: 0.3535396	total: 15.7s	remaining: 12m 50s
4:	learn: 0.3528650	total: 17.3s	remaining: 11m 13s
5:	learn: 0.3525553	total: 18.6s	remaining: 10m 2s
6:	learn: 0.3523122	total: 20s	remaining: 9m 11s
7:	learn: 0.3516731	total: 21.5s	remaining: 8m 36s
8:	learn: 0.3515235	total: 22.9s	remaining: 8m 5s
9:	learn: 0.3514549	total: 24.2s	remaining: 7m 39s
10:	learn: 0.3510128	total: 25.6s	remaining: 7m 19s
11:	learn: 0.3479715	total: 27.2s	remaining: 7m 6s
12:	learn: 0.3478300	total: 28.6s	remaining: 6m 51s
13:	learn: 0.3476107	total: 30s	remaining: 6m 38s
14:	learn: 0.3472826	total: 31.6s	remaining: 6m 30s
15:	learn: 0.3471666	total: 32.7s	remaining: 6m 15s
16:	learn: 0.3469759	total: 33.9s	remaining: 6m 4s
17:	learn: 0.3468908	total: 35.2s	remaining: 5m 56s
18:	learn: 0.3468600	total: 36.6s	remaining: 5m 48s
19:	learn: 0.3467592	tot

156:	learn: 0.3439787	total: 3m 36s	remaining: 59.3s
157:	learn: 0.3439707	total: 3m 37s	remaining: 57.9s
158:	learn: 0.3439690	total: 3m 39s	remaining: 56.5s
159:	learn: 0.3439624	total: 3m 40s	remaining: 55.1s
160:	learn: 0.3439544	total: 3m 41s	remaining: 53.7s
161:	learn: 0.3439482	total: 3m 43s	remaining: 52.3s
162:	learn: 0.3439406	total: 3m 44s	remaining: 51s
163:	learn: 0.3439364	total: 3m 45s	remaining: 49.6s
164:	learn: 0.3439312	total: 3m 46s	remaining: 48.1s
165:	learn: 0.3439205	total: 3m 48s	remaining: 46.7s
166:	learn: 0.3439118	total: 3m 49s	remaining: 45.3s
167:	learn: 0.3438897	total: 3m 50s	remaining: 43.9s
168:	learn: 0.3438845	total: 3m 51s	remaining: 42.5s
169:	learn: 0.3438697	total: 3m 53s	remaining: 41.2s
170:	learn: 0.3438645	total: 3m 54s	remaining: 39.8s
171:	learn: 0.3438594	total: 3m 55s	remaining: 38.4s
172:	learn: 0.3438442	total: 3m 57s	remaining: 37s
173:	learn: 0.3438285	total: 3m 58s	remaining: 35.6s
174:	learn: 0.3438184	total: 3m 59s	remaining: 34.

In [61]:
### Сохраним модель

catboost.save_model(
    'catboost_model',
    format="cbm"                  
)